In [5]:
import numpy as np
import matplotlib.pyplot as plt
from tfim import TFIMReservoir
from reservoir_graph_generation import initialize_graph, expand_graph, get_adjacency_matrix

In [22]:
N_in = 2

# Graph A: no expansion
G0, out0 = initialize_graph(N_in)
A0, nodes0 = get_adjacency_matrix(G0)  # A0: (N0 x N0)

# Graph B: one expansion step (mutates a fresh graph)
G1, out1 = initialize_graph(N_in)  # start fresh
G1 = expand_graph(G1, out1, steps=1)
A1, nodes1 = get_adjacency_matrix(G1)

adjacency_matrices = [A0.astype(float), A1.astype(float)]
nodes_lists        = [nodes0,           nodes1]

for k, (A, nodes) in enumerate(zip(adjacency_matrices, nodes_lists)):
    print(f"[{k}] nodes={len(nodes)} edges={int(A.sum()/2)} names={nodes}")

[0] nodes=3 edges=2 names=['I0', 'I1', 'O1']
[1] nodes=5 edges=5 names=['I0', 'I1', 'O1_1_1', 'O1_1_2', 'O1_1_3']


In [24]:
def indices_of_inputs(nodes):
    return [i for i, name in enumerate(nodes) if str(name).startswith("I")]

reservoirs = []
params = dict(J=0.8, h=1.0, dt=0.2, g_in=0.4, M=1)

for A, nodes in zip(adjacency_matrices, nodes_lists):
    N = A.shape[0]
    input_nodes = indices_of_inputs(nodes)
    if not input_nodes:
        raise RuntimeError("Did not find any nodes starting with 'I' in this graph.")
    # W_in: (M,N) zeros except on input columns
    W_in = np.zeros((params["M"], N), dtype=float)
    W_in[:, input_nodes] = params["g_in"]
    res = TFIMReservoir.from_adjacency(
        adj=A, J=params["J"], h=params["h"],
        W_in=W_in, input_nodes=input_nodes, dt=params["dt"], c_ops=None
    )
    reservoirs.append(dict(res=res, nodes=nodes, input_nodes=input_nodes))

print(f"Built {len(reservoirs)} reservoirs.")
for k, r in enumerate(reservoirs):
    print(f"[{k}] N={r['res'].N} input_nodes={r['input_nodes']}")

Built 2 reservoirs.
[0] N=3 input_nodes=[0, 1]
[1] N=5 input_nodes=[0, 1]


In [25]:
J, h, dt = 0.8, 1.0, 0.2
M, g_in = 1, 0.4

reservoirs = []
for A, nodes in zip(adjacency_matrices, nodes_lists):
    N = A.shape[0]
    input_nodes = [i for i, name in enumerate(nodes) if str(name).startswith("I")]
    W_in = np.zeros((M, N))
    W_in[:, input_nodes] = g_in
    res = TFIMReservoir.from_adjacency(A, J=J, h=h, W_in=W_in, input_nodes=input_nodes, dt=dt)
    reservoirs.append(res)
    print(f"Built reservoir: N={N}, inputs={input_nodes}")


Built reservoir: N=3, inputs=[0, 1]
Built reservoir: N=5, inputs=[0, 1]


In [ ]:
import numpy as np
from qutip import Qobj, basis, tensor, qeye, sigmax, sigmay, sigmaz
from qutip.qip.operations import cnot
from tfim import TFIMReservoir  # your class

# 2-qubit Pauli tensors in a fixed order; we will drop I⊗I to get 15 coords
I, X, Y, Z = qeye(2), sigmax(), sigmay(), sigmaz()
pauli2_all = [tensor(p,q) for p in [I,X,Y,Z] for q in [I,X,Y,Z]]
pauli2_all = pauli2_all[1:] # Dropping the first element since it's the identity
# pauli2_15  = [P for P in pauli2_all if not P.isidentity]

In [33]:
def rand_states_2q(n, seed=0):
    rng = np.random.default_rng(seed)
    vecs = []
    for _ in range(n):
        z = rng.normal(size=4) + 1j*rng.normal(size=4)
        z /= np.linalg.norm(z)
        vecs.append(Qobj(z, dims=[[2,2],[1,1]]))
    return vecs

def pauli15_expectations(psi):
    return np.array([ (psi.dag()*P*psi).full().item().real for P in pauli2_all ])

def make_dataset_cnot(n_train=1000, n_test=200, seed=1):
    train = rand_states_2q(n_train, seed)
    test  = rand_states_2q(n_test,  seed+1)
    def pack(psis):
        C_in  = np.stack([pauli15_expectations(psi) for psi in psis], axis=0)         # (N,15)
        C_out = np.stack([pauli15_expectations(cnot()*psi) for psi in psis], axis=0)  # (N,15)
        return C_in, C_out
    return pack(train) + pack(test)

In [34]:
# Pick which graph to use
A = adjacency_matrices[1]  # e.g., index 1 = after one expansion

# Drive all qubits for simplicity
N = A.shape[0]
M = 15          # 15 input channels for the 15 Pauli-string expectations
g_in = 0.4      # input→Z coupling scale
J, h, dt = 0.8, 1.0, 0.2

W_in = np.zeros((M, N)); W_in[:, :] = g_in  # drive all sites equally
res  = TFIMReservoir.from_adjacency(adj=A, J=J, h=h, W_in=W_in,
                                    input_nodes=list(range(N)), dt=dt, c_ops=None)

In [ ]:
def ridge_fit(X, Y, lam=1e-2):
    # X:(N,D), Y:(N,K) → W:(D+1,K) with bias
    Xb = np.hstack([X, np.ones((X.shape[0],1))])
    A  = Xb.T @ Xb + lam*np.eye(Xb.shape[1])
    return np.linalg.solve(A, Xb.T @ Y)

def run_encode_constant(res, U_in, T=60, verbose=True):
    # U_in: (N_samples, M). For each sample, hold u constant for T steps.
    X = []
    N_samples = U_in.shape[0]
    for idx, u in enumerate(U_in):
        U_seq = np.tile(u, (T,1))
        _, Phi = res.evolve(U_seq, collect_features=True)
        X.append(Phi[-1])  # last-time features (N,)
        # Progress print
        if verbose and (idx+1) % 10 == 0:  # every 100 samples
            print(f"  Encoded {idx+1}/{N_samples} samples")
    return np.stack(X, axis=0)  # (N_samples, N_qubits)

# Build data
print("Generating training and test data...")
Ctr_in, Ctr_out, Cte_in, Cte_out = make_dataset_cnot(n_train=1000, n_test=200, seed=3)
print(f"Train samples: {Ctr_in.shape[0]}, Test samples: {Cte_in.shape[0]}")

# Encode inputs, get reservoir features
T_steps = 60
print(f"\nEncoding training inputs for {T_steps} steps...")
Xtr = run_encode_constant(res, Ctr_in, T=T_steps)   # (Ntr, Nq)
print("  Done. Training features shape:", Xtr.shape)

print(f"Encoding test inputs for {T_steps} steps...")
Xte = run_encode_constant(res, Cte_in, T=T_steps)   # (Nte, Nq)
print("  Done. Test features shape:", Xte.shape)

# Fit linear readout to map features → 15 targets
print("\nFitting ridge regression readout...")
W = ridge_fit(Xtr, Ctr_out, lam=5e-2)               # (Nq+1, 15)
print("  Done. Readout weight matrix shape:", W.shape)

# Predict on test
print("\nPredicting on test set...")
Xte_b = np.hstack([Xte, np.ones((Xte.shape[0],1))])
Cte_pred = Xte_b @ W                                 # (Nte, 15)
rmse = float(np.sqrt(np.mean((Cte_pred - Cte_out)**2)))
print(f"Test RMSE on 15 expectations = {rmse:.4f}")

Generating training and test data...
Train samples: 1000, Test samples: 200

Encoding training inputs for 60 steps...


KeyboardInterrupt: 